In [2]:
import netCDF4 as nc
import numpy as np
import datetime
import rasterio
import glob
import rioxarray
import xarray
import os
import glob

# List of NetCDF file paths
# nc_files = [r'etincr_monthly.nc']  # Add your file paths here
nc_files = glob.glob(r'/home/komlavi/CWA_model/SRO_monthly_data_extraction/netcdf/*.nc')
output_folder = r"/home/komlavi/CWA_model/SRO_monthly_data_extraction/outdata"
# Give reference image for profile
ref_img = r"/home/komlavi/CWA_model/SRO_monthly_data_extraction/ref_img/AFR_Incremental_ET_M_ETincr_yearly_avg_N.tif"
x =0
for i in nc_files:
    print(i, x)
    x += 1

Season = "AFR" #change this for appropriate season 
    
# start_month = 3 #change this for appropriate months 
# end_month = 5
# start_month1 = 3
# end_month1 = 5


# Ensure the output folder exists; create it if necessary
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
# Loop through the NetCDF files
for nc_file in nc_files:
    nc_file = nc.Dataset(nc_file)
    time_var = nc_file.variables['time']
    units = time_var.units
    # Extract the reference date
    
    var_names = nc_file.variables.keys()
    print(var_names)

    # Specify the target variable names
    target_var_names = ['P', 'AET', 'Incremental_ET_M', 'Rainfall_ET_M', 'SRO_M' ]

    # Initialize the selected_variable variable as None
    selected_variable = None

    # Loop through the variable names and find the first match
    for var_name in var_names:
        if var_name in target_var_names:
            selected_variable = var_name
            break
            
    if selected_variable == 'P':
        reference_date = datetime.datetime.strptime(units, "days since %Y-%m-%d")
#         reference_date = datetime.datetime.strptime(units, "days since %Y-%m-%d %H:%M")
    if selected_variable == 'AET':
        reference_date = datetime.datetime.strptime(units, "days since %Y-%m-%d")
#         reference_date = datetime.datetime.strptime(units, "days since %Y-%m-%d %H:%M:%S")
    if selected_variable == 'Incremental_ET_M':
        reference_date = datetime.datetime.strptime(units, "days since %Y-%m-%d")
    if selected_variable == 'Rainfall_ET_M':
        reference_date = datetime.datetime.strptime(units, "days since %Y-%m-%d")
    if selected_variable == 'SRO_M':
        reference_date = datetime.datetime.strptime(units, "days since %Y-%m-%d %H:%M")
        
    
    if selected_variable == 'P':
        file_name = f"P_{Season}"
    if selected_variable == 'AET':
        file_name = f"ET_{Season}"
    if selected_variable == 'Incremental_ET_M':
        file_name = f"ETincr_{Season}"
    if selected_variable == 'Rainfall_ET_M':
        file_name = f"ETrain_{Season}"
    if selected_variable == 'SRO_M':
        file_name = f"sro_{Season}"
#     reference_date = datetime.datetime.strptime(units, "days since %Y-%m-%d %H:%M")
#     reference_date = datetime.datetime.strptime(units, "days since %Y-%m-%d")
    # Convert time values to datetime objects
    time_values = time_var[:]
    start_date = reference_date + datetime.timedelta(days=float(time_values[0]))
    print("Starting Year:",start_date.year)
    end_date = reference_date + datetime.timedelta(days=float(time_values[-1]))
    print("Ending Year:",end_date.year)
    # Find the indices corresponding to the first 6 months of every year

    for year in range(int(start_date.year), int(end_date.year) + 1):
        for Month in range(1,13):
            indices = []
            for i, date in enumerate(time_values):
                dt = reference_date + datetime.timedelta(days=float(date))
                if ((year == dt.year) and (Month == dt.month)):
                    indices.append(i)
#                     if ((start_month <= dt.month <= end_month) or (start_month1 <= dt.month <= end_month1)):
                        # print(dt.year)
                        # print(dt.month)
            
            data = nc_file.variables[selected_variable][indices, :, :]  # Replace 'your_variable_name' with the actual variable name
            sum_data = np.sum(data, axis=0)
            file_path = output_folder
            print(year)
            file_name1 = f"{file_name}_{year}.{Month}.tif"
    #     file_name = file_name + "_avg.tif"

        # Concatenate the directory path and file name (without extension) to create the full file path
            full_file_path = os.path.join(file_path, file_name1)
            print(full_file_path)

            dataset = rasterio.open(ref_img)
            output_profile = dataset.profile
            output_profile.update(count=1, dtype=rasterio.float32)

            with rasterio.open(full_file_path, 'w', **output_profile) as dst:
                dst.write(sum_data.astype(rasterio.float32), 1)
       
    print("GeoTIFF files are created:", file_name)


/home/komlavi/CWA_model/SRO_monthly_data_extraction/netcdf/AFR_SRO_M_SRO_monthly.nc 0
dict_keys(['time', 'latitude', 'longitude', 'SRO_M'])
Starting Year: 2003
Ending Year: 2021
2003
/home/komlavi/CWA_model/SRO_monthly_data_extraction/outdata/sro_AFR_2003.1.tif
2003
/home/komlavi/CWA_model/SRO_monthly_data_extraction/outdata/sro_AFR_2003.2.tif
2003
/home/komlavi/CWA_model/SRO_monthly_data_extraction/outdata/sro_AFR_2003.3.tif
2003
/home/komlavi/CWA_model/SRO_monthly_data_extraction/outdata/sro_AFR_2003.4.tif
2003
/home/komlavi/CWA_model/SRO_monthly_data_extraction/outdata/sro_AFR_2003.5.tif
2003
/home/komlavi/CWA_model/SRO_monthly_data_extraction/outdata/sro_AFR_2003.6.tif
2003
/home/komlavi/CWA_model/SRO_monthly_data_extraction/outdata/sro_AFR_2003.7.tif
2003
/home/komlavi/CWA_model/SRO_monthly_data_extraction/outdata/sro_AFR_2003.8.tif
2003
/home/komlavi/CWA_model/SRO_monthly_data_extraction/outdata/sro_AFR_2003.9.tif
2003
/home/komlavi/CWA_model/SRO_monthly_data_extraction/outdata/s

In [ ]:
import netCDF4 as nc
import numpy as np
import datetime
import rasterio
import glob
import rioxarray
import xarray
import os
import glob

# List of NetCDF file paths
# nc_files = [r'etincr_monthly.nc']  # Add your file paths here
nc_files = glob.glob(r'C:\Users\t.perera\Downloads\Sandra\*.nc')
output_folder = r"C:\Users\t.perera\OneDrive - CGIAR\Documents\Python Scripts\Rift_valley\Monthly_outputs\year_wise\example\wet2"
# Give reference image for profile
ref_img = r"C:\Users\t.perera\OneDrive - CGIAR\Documents\Python Scripts\Rift_valley\Yearly_outputs\chirps_ssebop\etincr_yearly_avg.tif"
x =0
for i in nc_files:
    print(i, x)
    x += 1

Season = "Wet1" #change this for appropriate season 
    
start_month = 3 #change this for appropriate months 
end_month = 5
start_month1 = 3
end_month1 = 5


# Ensure the output folder exists; create it if necessary
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
# Loop through the NetCDF files
for nc_file in nc_files:
    nc_file = nc.Dataset(nc_file)
#     time_var = nc_file.variables['time']
#     units = time_var.units
    # Extract the reference date
    
    var_names = nc_file.variables.keys()
    print(var_names)

    # Specify the target variable names
#     target_var_names = ['P', 'SSEBop', 'Incremental_ET_M', 'Rainfall_ET_M', 'SRO_M' ]

#     # Initialize the selected_variable variable as None
#     selected_variable = None

#     # Loop through the variable names and find the first match
#     for var_name in var_names:
#         if var_name in target_var_names:
#             selected_variable = var_name
#             break
#     reference_date = datetime.datetime.strptime(units, "days since %Y-%m-%d %H:%M:%S")
            
#     if selected_variable == 'P':
# #         reference_date = datetime.datetime.strptime(units, "days since %Y-%m-%d")
#         reference_date = datetime.datetime.strptime(units, "days since %Y-%m-%d %H:%M")
#     if selected_variable == 'SSEBop':
# #         reference_date = datetime.datetime.strptime(units, "days since %Y-%m-%d")
#         reference_date = datetime.datetime.strptime(units, "days since %Y-%m-%d %H:%M:%S")
#     if selected_variable == 'Incremental_ET_M':
#         reference_date = datetime.datetime.strptime(units, "days since %Y-%m-%d")
#     if selected_variable == 'Rainfall_ET_M':
#         reference_date = datetime.datetime.strptime(units, "days since %Y-%m-%d")
        
    
#     if selected_variable == 'P':
#         file_name = f"P_{Season}"
#     if selected_variable == 'SSEBop':
#         file_name = f"ET_{Season}"
#     if selected_variable == 'Incremental_ET_M':
#         file_name = f"ETincr_{Season}"
#     if selected_variable == 'Rainfall_ET_M':
#         file_name = f"ETrain_{Season}"
#     if selected_variable == 'SRO_M':
#         file_name = f"sro_{Season}"
#     reference_date = datetime.datetime.strptime(units, "days since %Y-%m-%d %H:%M")
#     reference_date = datetime.datetime.strptime(units, "days since %Y-%m-%d")
    # Convert time values to datetime objects
#     time_values = time_var[:]
#     start_date = reference_date + datetime.timedelta(days=float(time_values[0]))
#     print("Starting Year:",start_date)
#     end_date = reference_date + datetime.timedelta(days=float(time_values[-1]))
#     print("Ending Year:",end_date)
    # Find the indices corresponding to the first 6 months of every year
#     for year in range(int(start_date.year), int(end_date.year) + 1):
#         indices = []
#         for i, date in enumerate(time_values):
#             dt = reference_date + datetime.timedelta(days=float(date))
#             if year == dt.year:
#                 if ((start_month <= dt.month <= end_month) or (start_month1 <= dt.month <= end_month1)):
#                     # print(dt.year)
#                     # print(dt.month)
#                     indices.append(i)
#         data = nc_file.variables[selected_variable][indices, :, :]  # Replace 'your_variable_name' with the actual variable name
#         sum_data = np.sum(data, axis=0)
#         file_path = output_folder
#         print(year)
#         file_name1 = str(year)+"_"+file_name+"_.tif"
# #     file_name = file_name + "_avg.tif"

#     # Concatenate the directory path and file name (without extension) to create the full file path
#         full_file_path = os.path.join(file_path, file_name1)
#         print(full_file_path)
    
#         dataset = rasterio.open(ref_img)
#         output_profile = dataset.profile
#         output_profile.update(count=1, dtype=rasterio.float32)

#         with rasterio.open(full_file_path, 'w', **output_profile) as dst:
#             dst.write(sum_data.astype(rasterio.float32), 1)
       
#     print("GeoTIFF files are created:", file_name)



In [ ]:
import xarray as xr
import rasterio
from rasterio.transform import from_origin

# Load the NetCDF file using xarray
nc_file_path = r"C:\Users\t.perera\Downloads\Sandra\Drought_historical_Hazard.nc"
ds = xr.open_dataset(nc_file_path)

# Extract 'lon', 'lat', and 'SDII' variables
lon = ds['lon']
lat = ds['lat']
sdii = ds['Dhazard']

# Get the resolution and extent of the data
resolution = abs(lon[1] - lon[0]).values
print(resolution)
extent = [lon.min().values, lon.max().values, lat.min().values, lat.max().values]

# Create GeoTIFF files for 'SDII' variable
tif_file_path = r"C:\Users\t.perera\Downloads\Sandra\Tif\Drought_historical_Hazard_code_1.tif"
with rasterio.open(
    tif_file_path,
    'w',
    driver='GTiff',
    height=sdii.shape[0],
    width=sdii.shape[1],
    count=1,
    dtype=str(sdii.dtype),
    crs='+proj=latlong',
    transform=from_origin(extent[0], extent[3], resolution, resolution),
) as dst:
    dst.write(sdii.values, 1)

# You can also create GeoTIFF files for 'lon' and 'lat' if needed
# Just replace 'sdii' with 'lon' or 'lat' and adjust file names accordingly

# Close the xarray dataset
ds.close()


In [ ]:
import xarray as xr
import rasterio
from rasterio.transform import from_origin

# Load the NetCDF file using xarray
nc_file_path = r"C:\Users\t.perera\Downloads\Sandra\Drought_ssp585_Hazard.nc"
ds = xr.open_dataset(nc_file_path)

# Extract 'lon', 'lat', and 'SDII' variables
lon = ds['lon']
lat = ds['lat']
sdii = ds['Dhazard']

# Load the reference image to get CRS, resolution, and origin
reference_image_path = r"C:\Users\t.perera\Downloads\Sandra\Tif\flood_historical_Hazard_Q.tif"
with rasterio.open(reference_image_path) as ref_src:
    crs = ref_src.crs
    resolution = ref_src.res
    origin = ref_src.transform[2], ref_src.transform[5]  # x, y

# Create GeoTIFF files for 'SDII' variable using reference information
tif_file_path = r"C:\Users\t.perera\Downloads\Sandra\Tif\Drought_ssp585_Hazard_code_2.tif"
with rasterio.open(
    tif_file_path,
    'w',
    driver='GTiff',
    height=sdii.shape[0],
    width=sdii.shape[1],
    count=1,
    dtype=str(sdii.dtype),
    crs=crs,
    transform=from_origin(origin[0], origin[1], resolution[0], resolution[1]),
) as dst:
    dst.write(sdii.values, 1)

# You can also create GeoTIFF files for 'lon' and 'lat' if needed
# Just replace 'sdii' with 'lon' or 'lat' and adjust file names accordingly

# Close the xarray dataset
ds.close()


In [ ]:
import netCDF4 as nc
import numpy as np
import datetime
import rasterio
import glob
import rioxarray
import xarray
import os
import glob

tif_files = glob.glob(r'C:\Users\t.perera\OneDrive - CGIAR\Documents\Python Scripts\Rift_valley\Monthly_outputs\year_wise\wet2\*_ET_wet2_avg_.tif')
x =0
for i in tif_files:
    print(i, x)
    x += 1
tif_files1 = glob.glob(r'C:\Users\t.perera\OneDrive - CGIAR\Documents\Python Scripts\Rift_valley\Monthly_outputs\year_wise\wet2\*_P_wet2_avg_.tif')
x =0
for i in tif_files1:
    print(i, x)
    x += 1

In [ ]:
# Specify the paths to your TIFF files
P_Avg_tif = tif_files1[i]
ET_Avg_tif = tif_files[i]

# file_path = r"C:\Users\t.perera\OneDrive - CGIAR\Documents\Python Scripts\for averaging\nc_files\output_montly_volta"

# Output TIFF file path for the difference
tif_files[i] = filename.split('_')

# Get the part containing the year
year_part = None
for part in tif_files[i]:
    if part.isdigit() and len(part) == 4:  # Assuming the year is a 4-digit number
        year_part = part
        break
output_tif = file_path+str(year_part)+"\Rift_Valley_Water_yield_wet1_avg.tif"
print(output_tif)

# Open the TIFF files using rasterio
with rasterio.open(P_Avg_tif) as p_avg_ds, rasterio.open(ET_Avg_tif) as et_avg_ds:
    # Read the data from the TIFF files as numpy arrays
    p_avg_array = p_avg_ds.read(1)
    et_avg_array = et_avg_ds.read(1)

    # Calculate the difference between the two arrays
    difference_array = p_avg_array - et_avg_array
    with rasterio.open(output_tif, 'w', **output_profile) as dst:
            # Write the modified data to the output TIF
        dst.write(difference_array.astype(rasterio.float32), 1)

print(f"Difference saved to {output_tif}")

In [ ]:
import rasterio

# Assuming tif_files1 and tif_files are lists of TIFF file paths

for i in range(11):  # Loop from 0 to 10 inclusive
    P_Avg_tif = tif_files1[i]
    ET_Avg_tif = tif_files[i]

    # file_path = r"C:\Users\t.perera\OneDrive - CGIAR\Documents\Python Scripts\for averaging\nc_files\output_montly_volta"
    filename = os.path.basename(ET_Avg_tif)
    tif_files_parts = filename.split('_')

    # Get the part containing the year
    year_part = None
    for part in tif_files_parts:
        if part.isdigit() and len(part) == 4:  # Assuming the year is a 4-digit number
            year_part = part
            break

    output_tif = file_path+"\Rift_Valley_"+str(year_part)+"_Water_yield_wet2_avg.tif"

    # Open the TIFF files using rasterio
    with rasterio.open(P_Avg_tif) as p_avg_ds, rasterio.open(ET_Avg_tif) as et_avg_ds:
        # Read the data from the TIFF files as numpy arrays
        p_avg_array = p_avg_ds.read(1)
        et_avg_array = et_avg_ds.read(1)

        # Calculate the difference between the two arrays
        difference_array = p_avg_array - et_avg_array

        # Assuming output_profile is defined somewhere in your code
        output_profile = p_avg_ds.profile

        with rasterio.open(output_tif, 'w', **output_profile) as dst:
            # Write the modified data to the output TIF
            dst.write(difference_array.astype(rasterio.float32), 1)

    print(f"Difference for index {i} saved to {output_tif}")


In [ ]:
import netCDF4 as nc

# Open the NetCDF file for reading
ncfile = nc.Dataset("C:\Users\t.perera\Downloads\Sandra\flood_historical_Hazard.nc", 'r')

# Extract the reference date attribute
reference_date_str = ncfile.getncattr('reference_date')

# Close the NetCDF file
ncfile.close()

# Parse the reference date string into a datetime object
reference_date = datetime.datetime.strptime(reference_date_str, '%Y-%m-%d %H:%M')

print(reference_date)


In [ ]:
import numpy as np
from netCDF4 import Dataset
import rasterio
from rasterio.transform import from_origin

# Replace these with your input and output file paths
netcdf_file_path = "Volta_P_MSWEP.nc"
output_tif_path = 'output.tif'

# Open the NetCDF file
nc = Dataset(netcdf_file_path, 'r')

# Assuming the time dimension is named 'time', replace it with the actual name if needed
time_var = nc.variables['time'][:]
start_year = np.min(time_var)
end_year = np.max(time_var)

print(start_year)

# Calculate the number of years in the dataset
num_years = end_year - start_year + 1

# Create an empty array to store the yearly sums
yearly_sums = []

# Iterate through the years and calculate the sum of the first 6 months for each year
for year in range(start_year, end_year + 1):
    # Extract the monthly data for the current year
    year_data = nc.variables['P'][time_var[:] // 100 == year]
    
    # Sum the first 6 months (assuming January to June)
    sum_6_months = np.sum(year_data[:6, :, :], axis=0)
    
    # Add the sum to the yearly_sums array
    yearly_sums[year - start_year, :, :] = sum_6_months

# Get the spatial information from the NetCDF file
lon = nc.variables['lon'][:]
lat = nc.variables['lat'][:]
transform = from_origin(lon[0], lat[-1], lon[1] - lon[0], lat[0] - lat[1])

# Create a GeoTIFF file and write the data
with rasterio.open(output_tif_path, 'w', driver='GTiff', width=lon.size, height=lat.size, count=1, dtype=np.float32,
                   crs='EPSG:4326', transform=transform) as dst:
    dst.write(yearly_sums, 1)

# Close the NetCDF file
nc.close()

print(f"GeoTIFF file saved to {output_tif_path}")



In [ ]:
import xarray as xr
import numpy as np
import rasterio
from rasterio.transform import from_origin

# Load the NetCDF file
nc_file = "Volta_P_MSWEP.tif"
ds = xr.open_dataset(nc_file)

# Assuming that 'time' is a dimension named 'time' and represents monthly data

# Initialize variables to store the calculated averages
averages = []

# Iterate over each year in the dataset
for year in np.unique(ds['time.year']):
    # Convert the time values to datetime objects
    time_values = pd.to_datetime(ds['time'].values)
    
    # Select the first 4 months and last 2 months for the current year
    months_to_average = ds.sel(time=slice(f"{year}-01-01", f"{year}-04-30")) + ds.sel(time=slice(f"{year}-11-01", f"{year}-12-31"))
    
    # Calculate the mean along the time dimension for the selected months
    yearly_average = months_to_average.mean(dim='time')
    
    # Append the yearly average to the list
    averages.append(yearly_average)

# Concatenate the yearly averages along the 'year' dimension
annual_average = xr.concat(averages, dim='year')

# Specify the output GeoTIFF file name
output_tif = "output.tif"

# Get the transformation and coordinate system information from the NetCDF file
transform = rasterio.transform.from_origin(ds.longitude.values[0], ds.latitude.values[-1], abs(ds.longitude.values[1] - ds.longitude.values[0]), abs(ds.latitude.values[1] - ds.latitude.values[0]))
crs = ds.crs

# Write the data to a GeoTIFF file with the same transformation and coordinate system
with rasterio.open(output_tif, 'w', driver='GTiff', height=annual_average.sizes['latitude'], width=annual_average.sizes['longitude'], count=1, dtype=str(annual_average.dtype), crs=crs, transform=transform) as dst:
    dst.write(annual_average.values, 1)

print(f"Annual average saved to {output_tif}")
